In [ ]:
# Импорт необходимых библиотек
import gc
import math
import random
import numpy as np
import copy
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, SimpleRNN, GRU, LSTM, Dropout, TimeDistributed, Input
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import  SGD, RMSprop, Adam
from functools import cmp_to_key
from google.colab import drive

drive.mount('/content/gdrive')

import sys
sys.path.append('/content/gdrive/My Drive/Colab Notebooks/10_nodes_new')

# Определение графа с рёбрами, начальной и конечной вершинами
edges = [[1, 2], [1, 5], [1, 8], [2, 3], [3, 4], [3, 8], [3, 9], [4, 10], [5, 6], [6, 7], [6, 8], [6, 9], [7, 10], [9, 10]]
number_edges = len(edges)  # Количество рёбер
number_steps = 4  # Количество шагов в последовательности
start_node = 1  # Начальная вершина
end_node = 10  # Конечная вершина

# Загрузка данных из CSV файлов
df = pd.read_csv('gdrive/My Drive/Colab Notebooks/10_nodes_new/data/10_nodes_LU.csv')
x = df.iloc[:,:number_edges].values  # Признаки
y = df.iloc[:,number_edges:].values  # Метки

df_hp = pd.read_csv('gdrive/My Drive/Colab Notebooks/10_nodes_new/data/10_nodes_LU_hp.csv')
x_hp = df_hp.iloc[:,:number_edges].values  # Признаки для гиперпараметров
y_hp = df_hp.iloc[:,number_edges:].values  # Метки для гиперпараметров

# Разделение данных на обучающий, тестовый и валидационный наборы
from sklearn.model_selection import train_test_split
x_main_train, x_main_tv, y_main_train, y_main_tv = train_test_split(x, y, test_size=2/5)
x_main_test, x_main_valid, y_main_test, y_main_valid = train_test_split(x_main_tv, y_main_tv, test_size=1/2)

x_hp_train, x_hp_tv, y_hp_train, y_hp_tv = train_test_split(x_hp, y_hp, test_size=2/5)
x_hp_test, x_hp_valid, y_hp_test, y_hp_valid = train_test_split(x_hp_tv, y_hp_tv, test_size=1/2)

# Стандартизация данных
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_main_train)
x_test = sc.transform(x_main_test)
x_valid = sc.transform(x_main_valid)

sc_hp = StandardScaler()
x_train_1 = sc_hp.fit_transform(x_hp_train)
x_test_1 = sc_hp.transform(x_hp_test)
x_valid_1 = sc_hp.transform(x_hp_valid)

# Расширение размерности данных для временных рядов
x_train = np.repeat(x_train[:, np.newaxis, : ], number_steps, axis=1)
x_test = np.repeat(x_test[:, np.newaxis, : ], number_steps, axis=1)
x_valid = np.repeat(x_valid[:, np.newaxis, : ], number_steps, axis=1)

x_train_1 = np.repeat(x_train_1[:, np.newaxis, : ], number_steps, axis=1)
x_test_1 = np.repeat(x_test_1[:, np.newaxis, : ], number_steps, axis=1)
x_valid_1 = np.repeat(x_valid_1[:, np.newaxis, : ], number_steps, axis=1)

# Изменение формы меток для соответствия размерности
y_train = np.reshape(y_main_train, (y_main_train.shape[0], number_steps, number_edges))
y_test = np.reshape(y_main_test, (y_main_test.shape[0], number_steps, number_edges))
y_valid = np.reshape(y_main_valid, (y_main_valid.shape[0], number_steps, number_edges))

y_train_1 = np.reshape(y_hp_train, (y_hp_train.shape[0], number_steps, number_edges))
y_test_1 = np.reshape(y_hp_test, (y_hp_test.shape[0], number_steps, number_edges))
y_valid_1 = np.reshape(y_hp_valid, (y_hp_valid.shape[0], number_steps, number_edges))

# Вывод формы данных
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape, x_valid.shape, y_valid.shape, x_train_1.shape, y_train_1.shape, x_test_1.shape, y_test_1.shape)

# Определение гиперпараметров структуры и обучения
# Гиперпараметры структуры: количество слоёв, количество единиц, dropout, тип ячейки
# Гиперпараметры обучения: скорость обучения, размер батча
HP_1 = [[1, 2, 3, 4, 5],
       [-1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, -1],
       [0.0, 0.1, 0.2, 0.3, 0.4, 0.5],
       [SimpleRNN, GRU, LSTM],
       [SGD, RMSprop, Adam],
       [0.0001, 0.001, 0.01, 0.1],
       [16, 32, 64, 128, 256, 512, 1024]]

# Создание списка количества вариантов гиперпараметров
t = []
for p in HP_1:
    t.append(len(p))

# Инициализация вспомогательных переменных
t1 = []
hp = []
for i in range(t[0]):
    t1.append(t[1])
    t1.append(t[2])
    hp.append(HP_1[1])
    hp.append(HP_1[2])

# Добавление гиперпараметров, начиная с третьего элемента
for i in range(3, len(HP_1)):
    t1.append(t[i])
    hp.append(HP_1[i])

# Определение квадратных корней количества вариантов гиперпараметров
t2 = [int(np.sqrt(itm)) for itm in t1]

# Расчёт общего количества гиперпараметров
n_hp = 2*t[0] + len(HP_1) - 3

# Вывод информации о гиперпараметрах
print(t, t1, t2, n_hp)

# Определение пользовательской функции точности
def custom_accuracy(y_true, y_pred):
    threshold = 0.5
    y_pred_binary = tf.cast(tf.greater_equal(y_pred, threshold), tf.float32)
    y_true = tf.cast(y_true, tf.float32)
    stepwise_correct = tf.reduce_all(tf.equal(y_true, y_pred_binary), axis=-1)
    sequencewise_correct = tf.reduce_all(stepwise_correct, axis=-1)
    acc = tf.reduce_mean(tf.cast(sequencewise_correct, tf.float32))
    return acc

# Создание callback для ранней остановки при обучении модели
callback = tf.keras.callbacks.EarlyStopping(monitor='val_custom_accuracy',
                                            min_delta=0.001, patience=10, verbose=1, mode='max',
                                            restore_best_weights=True)

Mounted at /content/gdrive
(600000, 4, 14) (600000, 4, 14) (200000, 4, 14) (200000, 4, 14) (200000, 4, 14) (200000, 4, 14) (6000, 4, 14) (6000, 4, 14) (2000, 4, 14) (2000, 4, 14)
[5, 22, 6, 3, 3, 4, 7] [22, 6, 22, 6, 22, 6, 22, 6, 22, 6, 3, 3, 4, 7] [4, 2, 4, 2, 4, 2, 4, 2, 4, 2, 1, 1, 2, 2] 14


In [ ]:
class Network:
    def __init__(self):
        # Инициализация сети: генерируем решение, инициализируем значения фитнеса
        self.gen = self.create_solution()
        self.fitness = 0

    def create_solution(self):
        # Создание решения (генерация набора гиперпараметров)
        gen = []
        for i in range(n_hp):
            gen.append(np.random.choice(list(range(t1[i]))))  # Выбираем случайный гиперпараметр для каждого элемента
        return gen

    def check_gen(self):
        # Проверка генов (гиперпараметров), чтобы убедиться, что они допустимы
        state = False
        for i in range(n_hp):
            if i % 2 == 0 and i < 2 * t[0] and hp[i][self.gen[i]] != -1:
                state = True  # Разрешено использовать этот набор гиперпараметров
                break
        return state

    def evaluate(self):
        # Оценка модели на основе текущего решения (набора гиперпараметров)
        if self.check_gen() == False:
            return 0
        else:
            input_shape = (number_steps, number_edges)
            inputs = Input(shape=input_shape)  # Входной слой
            x_input = inputs
            cell_type = hp[2 * t[0]][self.gen[2 * t[0]]]  # Определяем тип ячейки (SimpleRNN, GRU или LSTM)

            # Построение рекуррентных слоёв на основе гиперпараметров
            for i in range(t[0]):
                if hp[2 * i][self.gen[2 * i]] != -1:
                    x_input = cell_type(hp[2 * i][self.gen[2 * i]], return_sequences=True)(x_input)  # Добавление рекуррентного слоя
                    x_input = Dropout(hp[2 * i + 1][self.gen[2 * i + 1]])(x_input)  # Dropout слой для предотвращения переобучения

            # Выходной слой (TimeDistributed) с сигмоидной активацией для прогнозирования по каждому шагу последовательности
            outputs = TimeDistributed(Dense(number_edges, activation="sigmoid"))(x_input)
            model = Model(inputs=inputs, outputs=outputs)  # Определение модели

            # Компиляция модели с выбранным оптимизатором и скоростью обучения
            opt = hp[2 * t[0] + 1][self.gen[2 * t[0] + 1]]
            model.compile(loss=BinaryCrossentropy(), optimizer=opt(learning_rate=hp[2 * t[0] + 2][self.gen[2 * t[0] + 2]]), metrics=[custom_accuracy])

            # Обучение модели на тренировочном наборе данных с использованием callback для ранней остановки
            model.fit(x_train_1, y_train_1, epochs=100, callbacks=[callback], validation_data=(x_valid_1, y_valid_1), batch_size=hp[2 * t[0] + 3][self.gen[2 * t[0] + 3]], verbose=0)

            # Прогнозирование и вычисление точности на тестовом наборе данных
            pred = model.predict(x_test_1)
            accuracy_hp = custom_accuracy(y_test_1, pred)
            accuracy_result = tf.keras.backend.get_value(accuracy_hp)
            acc = max(int(round(accuracy_result * 100)), 1)  # Округляем точность и гарантируем, что значение не меньше 1

            # Очистка сессии Keras для предотвращения утечек памяти
            tf.keras.backend.clear_session()
            return acc

In [ ]:
class GA:
    def __init__(self, N, Max, Pc, Pm, Ts):
        # Инициализация генетического алгоритма (GA)
        self.N = N  # Количество особей в популяции
        self.Max = Max  # Максимальное количество итераций
        self.Pc = Pc  # Вероятность скрещивания
        self.Pm = Pm  # Вероятность мутации
        self.Ts = Ts  # Размер турнира для отбора
        self.population = [Network() for i in range(self.N)]  # Инициализация популяции
        self.best = Network()  # Лучшая сеть
        self.pool = []  # Пул уже проверенных решений

    def population_fitness(self):
        # Вычисление фитнеса для всей популяции
        for i in range(len(self.population)):
            self.calculate_fitness(self.population[i])
            self.show(self.population[i])
        print("__________________________________________________________________")

    def calculate_fitness(self, new_mem):
        # Вычисление фитнеса для нового решения
        cond = True
        for mem in self.pool:
            if tuple(new_mem.gen) == tuple(mem.gen):
                cond = False
                new_mem.fitness = mem.fitness
                break
        if cond == True:
            new_mem.fitness = new_mem.evaluate()
            self.pool.append(copy.deepcopy(new_mem))

    def complexity(self, m):
        # Вычисление сложности модели
        coeff = None
        r_a = 0
        if m.gen[2 * t[0]] == 0:
            coeff = 1  # SimpleRNN
        elif m.gen[2 * t[0]] == 1:
            coeff = 3  # GRU
            r_a = 1
        else:
            coeff = 4  # LSTM
        a = 0
        g = number_edges
        for i in range(t[0]):
            if hp[2 * i][m.gen[2 * i]] != -1:
                b = coeff * hp[2 * i][m.gen[2 * i]] * (hp[2 * i][m.gen[2 * i]] + g + r_a + 1)
                a += b
                g = hp[2 * i][m.gen[2 * i]]
        a += number_edges * (g + 1)
        return a

    def compare(self, m1, m2):
        # Сравнение двух решений на основе их фитнеса и сложности
        if m1.fitness == m2.fitness:
            a = self.complexity(m1)
            b = self.complexity(m2)
            if a < b:
                return -1
            elif a == b:
                return 0
            else:
                return 1
        elif m1.fitness > m2.fitness:
            return -1
        else:
            return 1

    def crossover(self):
        # Скрещивание между парами родителей
        children = []
        for i in range(0, self.N, 2):
            father = np.random.randint(len(self.population))
            mother = np.random.randint(len(self.population))
            parents_1 = copy.deepcopy(self.population[father])
            parents_2 = copy.deepcopy(self.population[mother])
            if np.random.uniform(0, 1) < self.Pc:
                # Маска для скрещивания
                mask = np.random.choice([0, 1], size=(n_hp,))
                for j in range(n_hp):
                    if mask[j] == 1:
                        # Обмен генов между родителями
                        temp = parents_1.gen[j]
                        parents_1.gen[j] = parents_2.gen[j]
                        parents_2.gen[j] = temp
                children.append(copy.deepcopy(parents_1))
                children.append(copy.deepcopy(parents_2))
            else:
                children.append(copy.deepcopy(parents_1))
                children.append(copy.deepcopy(parents_2))
        self.population = copy.deepcopy(children)

    def mutation(self):
        # Мутация в популяции
        for i in range(len(self.population)):
            if np.random.uniform(0, 1) < self.Pm:
                print("Mutation at: i = ", i)
                mask = np.random.choice([0, 1], size=(n_hp,))
                for j in range(n_hp):
                    if mask[j] == 1:
                        # Случайная замена гена
                        self.population[i].gen[j] = np.random.choice(list(range(t1[j])))

    def selection(self):
        # Отбор по турниру
        print("Selection:")
        new_population = []
        population_size = len(self.population)
        for _ in range(self.N):
            current_ts = min(self.Ts, population_size)  # Размер турнира
            tournament_indices = np.random.choice(population_size, current_ts, replace=False)  # Выбор участников турнира
            tournament = [self.population[i] for i in tournament_indices]
            winner = min(tournament, key=cmp_to_key(self.compare))  # Победитель турнира
            new_population.append(copy.deepcopy(winner))
        self.population = new_population

    def current_population(self):
        # Отображение текущей популяции
        for i in range(len(self.population)):
            self.show(self.population[i])
        print("__________________________________________________________________")

    def get_best(self):
        # Получение лучшего решения
        self.population.sort(key=cmp_to_key(self.compare))
        self.current_population()
        if self.compare(self.best, self.population[0]) == 1:
            self.best = copy.deepcopy(self.population[0])  # Обновляем лучшее решение
        self.show(self.best)
        print("__________________________________________________________________")

    def show(self, m):
        # Отображение информации о решении
        p_copy = []
        for j in range(t[0]):
            p_copy.append((m.gen[2 * j], m.gen[2 * j + 1]))
        p_copy.append(m.gen[2 * t[0]:])
        print(p_copy, m.fitness)

    def do(self):
        # Основной цикл выполнения генетического алгоритма (GA)
        for itr in range(self.Max):
            print("iteration = ", itr)
            self.crossover()  # Выполняем скрещивание
            self.current_population()  # Отображение популяции
            self.mutation()  # Выполняем мутацию
            self.current_population()  # Отображение популяции
            self.population_fitness()  # Вычисляем фитнес популяции
            self.selection()  # Отбор лучшей популяции
            self.get_best()  # Получаем лучшее решение
            gc.collect()  # Очистка памяти

In [ ]:
# N = 10
# Max = 10
# Pc = 0.7
# Pm = 0.1
# Ts = 2

In [ ]:
# import time
# for _ in range(10):
#     start = time.time()
#     alg = GA(N, Max, Pc, Pm, Ts)
#     alg.do()
#     with open('/content/gdrive/My Drive/Colab Notebooks/10_nodes_new/GA/acc/GA_3.txt', 'a') as file_1:
#         file_1.write(", ".join(map(str, alg.best.gen)) + '\n')
#     end = time.time()
#     run_time = int(end - start)
#     with open('/content/gdrive/My Drive/Colab Notebooks/10_nodes_new/GA/acc/GA_time_3.txt', 'a') as file_2:
#         file_2.write(f"{run_time}\n")

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
Mutation at: i =  8
[(6, 2), (18, 1), (6, 3), (1, 2), (2, 1), [2, 0, 1, 1]] 0
[(13, 1), (8, 3), (7, 4), (17, 4), (19, 4), [0, 1, 1, 2]] 0
[(6, 0), (20, 1), (3, 0), (4, 5), (17, 1), [2, 2, 1, 4]] 0
[(20, 4), (13, 4), (15, 3), (11, 0), (0, 0), [1, 0, 2, 3]] 0
[(16, 1), (8, 1), (6, 4), (7, 4), (0, 1), [1, 1, 2, 1]] 0
[(13, 2), (11, 0), (12, 2), (17, 5), (8, 3), [1, 0, 1, 2]] 0
[(20, 4), (20, 1), (3, 0), (4, 5), (0, 0), [2, 0, 2, 4]] 0
[(20, 4), (20, 1), (3, 0), (4, 5), (0, 0), [2, 0, 2, 4]] 0
[(17, 1), (8, 5), (17, 2), (12, 5), (1, 4), [2, 1, 1, 6]] 0
[(12, 3), (8, 4), (1, 1), (21, 1), (9, 1), [1, 2, 0, 4]] 0
__________________________________________________________________
Restoring model weights from the end of the best epoch: 1.
Epoch 11: early stopping
63/63 [==============================] - 2s 4ms/step
[(6, 2), (18, 1), (6, 3), (1, 2), (2, 1), [2, 0, 1, 1]] 1
63/63 [==============================] - 1s 4ms/step
[(13, 1), (8, 

In [ ]:
# input_shape = (number_steps, number_edges)
# inputs = Input(shape=input_shape)
# x_input = inputs
# cell_type = hp[2*t[0]][alg.best.gen[2*t[0]]]
# for i in range(t[0]):
#     if hp[2*i][alg.best.gen[2*i]] != -1:
#         x_input = cell_type(hp[2*i][alg.best.gen[2*i]], return_sequences=True)(x_input)
#         x_input = Dropout(hp[2*i+1][alg.best.gen[2*i+1]])(x_input)
# outputs = TimeDistributed(Dense(number_edges, activation="sigmoid"))(x_input)
# model = Model(inputs=inputs, outputs=outputs)
# opt = hp[2*t[0]+1][alg.best.gen[2*t[0]+1]]
# model.compile(loss=BinaryCrossentropy(), optimizer=opt(learning_rate=hp[2*t[0]+2][alg.best.gen[2*t[0]+2]]), metrics = [custom_accuracy])
# model.summary()
# print(alg.complexity(alg.best))
# print(opt)
# history = model.fit(x_train, y_train, epochs=1000, batch_size=hp[2*t[0]+3][alg.best.gen[2*t[0]+3]], callbacks=[callback], validation_data=(x_valid, y_valid), verbose=1)

In [ ]:
# from Results import Results
# pred = model.predict(x_train)
# pred_test = model.predict(x_test)
# r = Results(number_edges, edges, start_node, end_node, number_steps)
# accuracy_train = r.get_accuracy(pred, x_main_train, y_train)
# accuracy_test = r.get_accuracy(pred_test, x_main_test, y_test)
# print(accuracy_train, accuracy_test)